<a href="https://colab.research.google.com/github/lnpetrova/comp_ling/blob/master/ML_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from IPython.display import Image

In [0]:
import tarfile
import os
import pandas as pd
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score, roc_curve

In [4]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2020-02-18 10:20:51--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  24.0MB/s    in 5.7s    

2020-02-18 10:20:57 (14.0 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [0]:
tarfile.open('/content/aclImdb_v1.tar.gz', 'r').extractall()

In [0]:
path_test_neg = '/content/aclImdb/test/neg'
path_test_pos = '/content/aclImdb/test/pos'
path_train_neg = '/content/aclImdb/train/neg'
path_train_pos = '/content/aclImdb/train/pos'

In [0]:
find_test_neg = os.walk(path_test_neg)
tests_neg = list(find_test_neg)[0][-1]
#print(tests_neg)

find_test_pos = os.walk(path_test_pos)
tests_pos = list(find_test_pos)[0][-1]

find_train_neg = os.walk(path_train_neg)
trains_neg = list(find_train_neg)[0][-1]

find_train_pos = os.walk(path_train_pos)
trains_pos = list(find_train_pos)[0][-1]

In [0]:
# pathes = []
# pathes.append(path_test_neg)
# pathes.append(path_test_pos)
# pathes.append(path_train_pos)
# pathes.append(path_train_neg)

In [0]:
# def get_files(path):
#   find_files = os.walk(path)
#   list_of_files = list(find_files)[0][-1]
#   return list_of_files

In [0]:
# for path in pathes:
#   get_files(path)

In [0]:
def collect_texts(path, files):
    texts = [open('{}/{}'.format(path, file)).read().lower() for file in files]
    return texts

In [47]:
test_pos_df = pd.DataFrame(collect_texts(path_test_pos, tests_pos),  columns = ['text'])
test_pos_df['polarity'] = 1
test_pos_df.head()

,text,polarity
0,i've enjoyed this movie ever since i was a kid...,1
1,"this was very funny, even if it fell apart a l...",1
2,this is the best show ever no matter what you ...,1
3,"to say the truth, i went to see this movie onl...",1
4,simon wests pg-13 thriller about a babysitter ...,1


In [49]:
test_neg_df = pd.DataFrame(collect_texts(path_test_neg, tests_neg),  columns = ['text'])
test_neg_df['polarity'] = 0
test_neg_df.head()

,text,polarity
0,derek jarman has shown us time and time again ...,0
1,"well, i remember when the studio sacked schrad...",0
2,i was loaned this dvd by the director of a fil...,0
3,"this movie was shot using a digital camera, an...",0
4,i sat last night to see this film being played...,0


In [50]:
test = pd.concat((test_pos_df, test_neg_df))
test

,text,polarity
0,i've enjoyed this movie ever since i was a kid...,1
1,"this was very funny, even if it fell apart a l...",1
2,this is the best show ever no matter what you ...,1
3,"to say the truth, i went to see this movie onl...",1
4,simon wests pg-13 thriller about a babysitter ...,1
...,...,...
12495,"like a lot of movies involving little kids, th...",0
12496,i love basketball and this seemed like an intr...,0
12497,i was very excited when paranormal state first...,0
12498,"... once. ""manos, the hands of fate."" that was...",0


In [51]:
train_pos_df = pd.DataFrame(collect_texts(path_train_pos, trains_pos),  columns = ['text'])
train_pos_df['polarity'] = 1
train_pos_df.head()

,text,polarity
0,there are many reasons to watch this movie: to...,1
1,if you ever see a stand up comedy movie this i...,1
2,"michelle rodrigez was made for this movie, whe...",1
3,edmund lowe (who reminds me somewhat of warren...,1
4,what an ambitious project kenneth branagh unde...,1


In [52]:
train_neg_df = pd.DataFrame(collect_texts(path_train_neg, trains_neg),  columns = ['text'])
train_neg_df['polarity'] = 0
train_neg_df.head()

,text,polarity
0,this film is justly famous as one of the most ...,0
1,definitely the worst movie i have ever seen......,0
2,crazed shotgun toting-incest driven-revenge se...,0
3,"it was a good story, but not very well told. i...",0
4,so far nightmares and dreamscapes has been err...,0


In [53]:
train = pd.concat((train_pos_df, train_neg_df))
train

,text,polarity
0,there are many reasons to watch this movie: to...,1
1,if you ever see a stand up comedy movie this i...,1
2,"michelle rodrigez was made for this movie, whe...",1
3,edmund lowe (who reminds me somewhat of warren...,1
4,what an ambitious project kenneth branagh unde...,1
...,...,...
12495,"i have seen many, many productions of the nutc...",0
12496,"okay, during this past thanksgiving break, whi...",0
12497,i just got the uk 4-disc special edition of su...,0
12498,this is loosely based on the ideas of the orig...,0


In [68]:
train = shuffle(train, random_state=42)
train = train[:1000]
train.head()

,text,polarity
1123,"rich, alcoholic robert stack falls in love wit...",1
11225,this is one of those horror flicks where twent...,0
1599,"along with the ""maratonci trce pocasni krug"" f...",1
8925,this movie had so much potential - a strong ca...,0
6580,"although the plot was a bit sappy at times, an...",1


In [70]:
X_train = vectorizer.fit_transform(train.text)
print(X_train)
y_train = train.polarity
print(y_train)

  (0, 10603)	0.06813185799949871
  (0, 5004)	0.03784479760805218
  (0, 16801)	0.07499387605718667
  (0, 3766)	0.0924097293736522
  (0, 16138)	0.08598205609838092
  (0, 17522)	0.1143860894560407
  (0, 17580)	0.08244465223143992
  (0, 14539)	0.08046049188588483
  (0, 13884)	0.0515969309083381
  (0, 412)	0.06335856155738169
  (0, 16003)	0.06304463856941929
  (0, 18098)	0.058024604183609385
  (0, 4573)	0.10339790941484646
  (0, 6085)	0.07499387605718667
  (0, 7487)	0.06400569601599242
  (0, 17822)	0.0668959654243227
  (0, 17888)	0.061847648082118775
  (0, 9178)	0.09452654817413314
  (0, 15060)	0.10339790941484646
  (0, 5696)	0.052370381516187434
  (0, 7232)	0.03680189134253006
  (0, 7019)	0.09697023613957517
  (0, 18287)	0.051150349465449836
  (0, 14394)	0.05757802274072113
  (0, 8737)	0.04870666150000781
  :	:
  (999, 17900)	0.10188019410828489
  (999, 8518)	0.1033129394717635
  (999, 12006)	0.0719021608590993
  (999, 13169)	0.07249707595392724
  (999, 14825)	0.08257542148984312
  (999, 7

In [69]:
test = shuffle(test, random_state=42)
test = test[:1000]
test.head()

,text,polarity
311,"first of all, blythe danner doesn't look anywh...",0
5742,a college professor is working on creating zom...,0
2760,sex lives of the potato men is about the sex l...,0
10494,"first off, i'm amazed to see that this film ha...",0
1045,joline (heather graham) sets out after her hus...,1


In [66]:
X_test = vectorizer.transform(test.text)
print(X_test)
y_test = test.polarity
print(y_test)

  (0, 50182)	0.25409307895967775
  (0, 46941)	0.15639587404492783
  (0, 46251)	0.21417445087627973
  (0, 45961)	0.058937567064647756
  (0, 44075)	0.1578856133116422
  (0, 43539)	0.27405239300137674
  (0, 40486)	0.17759122347967154
  (0, 40465)	0.12476110974871775
  (0, 40462)	0.10221909092568313
  (0, 36884)	0.06277443041656679
  (0, 34423)	0.10266496104769067
  (0, 34405)	0.10545440615309042
  (0, 30459)	0.15497943095877356
  (0, 30260)	0.0428151163748678
  (0, 29373)	0.10404555478024789
  (0, 27099)	0.08665252683080375
  (0, 26986)	0.09014090398888251
  (0, 26121)	0.11684251771012187
  (0, 26018)	0.4265238368760321
  (0, 25403)	0.12676506274048152
  (0, 24686)	0.05393605941586017
  (0, 22066)	0.11927015798764942
  (0, 21802)	0.19915741646930435
  (0, 19282)	0.08656676294495368
  (0, 19062)	0.11200389844483977
  :	:
  (2499, 16105)	0.1385676992807104
  (2499, 16060)	0.1446054195719868
  (2499, 14590)	0.14240354348277062
  (2499, 14476)	0.07890168645017724
  (2499, 12646)	0.16505558217

## **Метод ближайшего соседа**

In [72]:
clf_KN = KNeighborsClassifier().fit(X_train_counts, test_data_short.Polarity)

NameError: ignored

In [59]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [60]:
logreg_preds = logreg.predict(X_test)
print(y_test.values)
print(logreg_preds)

[1 0 1 ... 0 0 1]
[1 0 1 ... 0 0 1]


In [61]:
eval_sklearn(y_test, logreg_preds)

NameError: ignored

In [0]:

y_true = np.array([0, 1, 1, 1, 0])
y_predicted = np.array([0.1, 0.7, 0.4, 0.2, 0.27])
fpr, tpr, _ = roc_curve(y_true, y_predicted)
pyplot.plot(fpr, tpr, marker='.', label='Test')
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

In [0]:
from sklearn.datasets import fetch_20newsgroups
data.target_names